In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)
x_train = train_datagen.flow_from_directory(r'C:/Users/prave/Downloads/Dataset/train', target_size=(64, 64),
                                            batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory(r'C:/Users/prave/Downloads/Dataset/test', target_size=(64, 64), batch_size=5,
                                          color_mode='grayscale', class_mode='categorical')
import numpy as np
import tensorflow as tf0
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, MaxPooling2D, Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=6, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(x_train, steps_per_epoch=len(x_train), epochs=25, validation_data=x_test,
                    validation_steps=len(x_test))
model.save('gesture.h5')
model_json = model.to_json()
with open("model-bw.json", "w") as json_file: json_file.write(model_json)
from tensorflow.keras.models import load_model
from keras.preprocessing import image

model = load_model("gesture.h5")
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model("gesture.h5")
path = r'C:/Users/prave/Downloads/Dataset/test/0/0.jpg'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

imgs = mpimg.imread(path)
imgplot = plt.imshow(imgs)
plt.show()
img = image.load_img(path, color_mode='grayscale', target_size=(64, 64))
x = image.img_to_array(img)  # image to array
x.shape
type(x)
x = np.expand_dims(x, axis=0)
x.shape
pred = (model.predict(x) > 0.5).astype("int64")
pred
index = ['0', '1', '2', '3', '4', '5']
index[np.argmax(pred[0])]
import numpy as np

p = []
for i in range(0, 6):
    for j in range(0, 5):
        path = r'C:/Users/prave/Downloads/Dataset/test/' + str(i) + "/" + str(j) + ".jpg"
        img = image.load_img(path, color_mode="grayscale", target_size=(64, 64))
        x = image.img_to_array(img)  # image to array
        x = np.expand_dims(x, axis=0)  # changing the shape
        pred = np.argmax(model.predict(x), axis=-1)  # predicting the classes
        p.append(pred)
print(p)
result = []
index = ['0', '1', '2', '3', '4', '5']
for i in p:
    result.append(index[i[0]])

print(result)
from flask import Flask, render_template, request
import operator
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras.models import load_model
import os
from werkzeug.utils import secure_filename

app = Flask(_name_, template_folder="templates")
model = load_model('gesture.h5')
print("Loaded Model From Disk")
app = Flask(_name_)


@app.route("/")
def home():
    return render_template("C:/Users/prave/Downloads/backup ibm/temphome")


@app.route('/intro')
def intro():
    return render_template("C:/Users/prave/Downloads/backup ibm/tempintroo")


@app.route('/image1', methods=['GET', 'POST'])
def image1():
    return render_template("C:/Users/prave/Downloads/backup ibm/templaunch")


@app.route('/predict', methods=['GET', 'POST'])
def launch():
    if request.method == 'POST':
        print("inside image")
        f = request.files['image']
        basepath = os.path.dirname(file)
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        print(file_path)
        cap = cv2.VideoCapture(0)
        while True:
            _, frame = cap.read()
            frame = cv2.flip(frame, 1)
            x1 = int(0.5 * frame.shape[1])
            y1 = 10
            x2 = frame.shape[1] - 10
            y2 = int(0.5 * frame.shape[1])
            cv2.rectangle(frame, (x1 - 1, y1 - 1), (x2 + 1, y2 + 1), (255, 0, 0), 1)
            roi = frame[y1:y2, x1:x2]
            roi = cv2.resize(roi, (64, 64))
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
            cv2.imshow("test", test_image)
            result = model.predict(test_image.reshape(1, 64, 64, 1))
            prediction = {'ZERO': result[0][0],
                          'ONE': result[0][1],
                          'TWO': result[0][2],
                          'THREE': result[0][3],
                          'FOUR': result[0][4],
                          'FIVE': result[0][5]}
            prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
            cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 1)
            cv2.imshow("Frame", frame)
            image1 = cv2.imread(file_path)
            if prediction[0][0] == 'ONE':
                resized = cv2.resize(image1, (200, 200))
                cv2.imshow("Fixed Resizing", resized)
                key = cv2.waitKey(3000)
                if (key & 0xFF) == ord("1"):
                    cv2.destroyWindow("Fixed Resizing")
            elif prediction[0][0] == 'ZERO':
                cv2.rectangle(image1, (480, 170), (650, 420), (0, 0, 255), 2)
                cv2.imshow("Rectangle", image1)
                cv2.waitKey(0)
                key = cv2.waitKey(3000)
                if (key & 0xFF) == ord("0"):
                    cv2.destroyWindow("Rectangle")
            elif prediction[0][0] == 'TWO':
                (h, w, d) = image1.shape
                center = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D(center, -45, 1.0)
                rotated = cv2.warpAffine(image1, M, (w, h))
                cv2.imshow("OpenCV Rotation", rotated)
                key = cv2.waitKey(3000)
                if (key & 0xFF) == ord("2"):
                    cv2.destroyWindow("OpenCV Rotation")
            elif prediction[0][0] == 'THREE':
                blurred = cv2.GaussianBlur(image1, (21, 21), 0)
                cv2.imshow("Blurred", blurred)
                key = cv2.waitKey(3000)
                if (key & 0xFF) == ord("3"):
                    cv2.destroyWindow("Blurred")
            elif prediction[0][0] == 'FOUR':
                resized = cv2.resize(image1, (400, 400))
                cv2.imshow("Fixed Resizing", resized)
                key = cv2.waitKey(3000)
                if (key & 0xFF) == ord("4"):
                    cv2.destroyWindow("Fixed Resizing")
            elif prediction[0][0] == 'FIVE':
                '''(h, w, d) = image1.shape
                center = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D(center, 45, 1.0)
                rotated = cv2.warpAffine(image1, M, (w, h))'''
                gray = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
                cv2.imshow("OpenCV Gray Scale", gray)
                key = cv2.waitKey(3000)
                if (key & 0xFF) == ord("5"):
                    cv2.destroyWindow("OpenCV Gray Scale")
            else:
                continue
            interrupt = cv2.waitKey(10)
            if interrupt & 0xFF == 27:  # esc key
                break
        cap.release()
        cv2.destroyAllWindows()
    return render_template("C:/Users/prave/Downloads/backup ibm/temphome")


if _name_ == '_main_':
    app.run(debug=False)